Github \
https://github.com/kohya-ss/sd-scripts/tree/main

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt autoremove
!sudo apt-get -y update
!sudo apt-get -y install python3.10

!sudo cp `which python3.10` /usr/local/bin/python

!wget https://bootstrap.pypa.io/get-pip.py
!sudo python get-pip.py

!python --version
!pip --version

In [ ]:
%cd '/content'
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd 'sd-scripts'

In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade -r requirements.txt
!pip install xformers==0.0.20

In [ ]:
!pip install protobuf==3.20.3
!pip install bitsandbytes==0.40.1

In [ ]:
!accelerate config default --mixed_precision fp16

In [ ]:
!nvidia-smi

In [ ]:
img_path = 'fashion-dataset/puma/topwear'
model_path = 'sd_xl_base_1.0.safetensors'
output_path = 'puma_topwear/lora'

In [ ]:
!python finetune/make_captions.py --batch_size 8 "$img_path"

In [ ]:
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 "$img_path"

In [ ]:
!python finetune/merge_captions_to_metadata.py "$img_path" meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py "$img_path" --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

In [ ]:
!python finetune/prepare_buckets_latents.py \
  "$img_path" \
  meta_clean.json \
  meta_lat.json \
  "$model_path" \
  --batch_size 4 \
  --max_resolution 1024,1024 \
  --mixed_precision no

In [ ]:
!cat meta_lat.json

In [ ]:
!accelerate launch  \
  --num_cpu_threads_per_process 1 sdxl_train_network.py \
  --pretrained_model_name_or_path="$model_path" \
  --in_json meta_lat.json \
  --train_data_dir="$img_path" \
  --output_dir="$output_path" \
  --shuffle_caption \
  --train_batch_size=1 \
  --learning_rate=1e-4 \
  --max_train_epochs=10 \
  --use_8bit_adam \
  --xformers \
  --gradient_checkpointing \
  --mixed_precision=fp16 \
  --save_every_n_epochs=5 \
  --save_precision=fp16 \
  --save_model_as=safetensors  \
  --network_module=networks.lora

In [ ]:
from google.colab import runtime
runtime.unassign()